In [1]:
# import
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
data_info = pd.read_csv('data_info.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
# 원본데이터 복사
df = train.copy()

In [4]:
# 잘 바뀌었는 지 확인
df.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [5]:
# 불필요하다고 생각되는 변수 제거
df.drop(['id','road_name','vehicle_restricted','base_date','start_node_name','end_node_name'],axis=1,inplace=True)
test.drop(['id','road_name','vehicle_restricted','base_date','start_node_name','end_node_name'],axis=1,inplace=True)

In [6]:
# 데이터 확인
df.head()

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,target
0,목,17,1,106,0,0,60.0,32400.0,0.0,3,33.427747,126.662612,없음,33.427749,126.662335,없음,52.0
1,목,21,2,103,0,0,60.0,0.0,0.0,0,33.500730,126.529107,있음,33.504811,126.526240,없음,30.0
2,일,7,2,103,0,0,80.0,0.0,0.0,0,33.279145,126.368598,없음,33.280072,126.362147,없음,61.0
3,금,13,2,107,0,0,50.0,0.0,0.0,0,33.246081,126.567204,없음,33.245565,126.566228,없음,20.0
4,화,8,2,103,0,0,80.0,0.0,0.0,0,33.462214,126.326551,없음,33.462677,126.330152,없음,38.0


In [7]:
# key = 'base_hour'
# tmp = df[key].tolist()

# for x in tqdm(range(len(tmp))):
#     y = df.columns.tolist().index(key)
#     if tmp[x] >=0 and tmp[x] <=3:
#         df.iloc[x,y] = 0
#     elif tmp[x] >=4 and tmp[x] <= 7:
#         df.iloc[x,y] = 1
#     elif tmp[x] >=8 and tmp[x] <= 11:
#         df.iloc[x,y] = 2
#     elif tmp[x] >=12 and tmp[x] <=15:
#         df.iloc[x,y] = 3
#     elif tmp[x] >=16 and tmp[x] <=19:
#         df.iloc[x,y] = 4
#     elif tmp[x] >=20 and tmp[x] <=23:
#         df.iloc[x,y] = 5

In [8]:
# 24시간을 4시간으로 자르고, 새로운 범주화 적용
# train set
key = 'base_hour'
tmp = list(set(df[key].tolist()))
result = pd.DataFrame()

for i in tqdm(range(len(tmp))):
    df_w = df[df[key]==tmp[i]]
    sample_num = df[key].tolist()[0]
    if sample_num >=0 and sample_num <=3:
        df_w[key] = 0 
    elif sample_num >=4 and sample_num <=7:
        df_w[key] = 1 
    elif sample_num >=8 and sample_num <=11:
        df_w[key] = 2
    elif sample_num >=12 and sample_num <=15:
        df_w[key] = 3
    elif sample_num >=16 and sample_num <=19:
        df_w[key] = 4
    elif sample_num >=20 and sample_num <=23:
        df_w[key] = 5
    
    result = pd.concat([result,df_w])
    
    


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:19<00:00,  1.23it/s]


In [9]:
# 24시간을 4시간으로 자르고, 새로운 범주화 적용
# test set

key = 'base_hour'
tmp = list(set(test[key].tolist()))
result1 = pd.DataFrame()

for i in tqdm(range(len(tmp))):
    df_one = test[test[key]==tmp[i]]
    sample_num = test[key].tolist()[0]
    if sample_num >=0 and sample_num <=3:
        df_one[key] = 0 
    elif sample_num >=4 and sample_num <=7:
        df_one[key] = 1 
    elif sample_num >=8 and sample_num <=11:
        df_one[key] = 2
    elif sample_num >=12 and sample_num <=15:
        df_one[key] = 3
    elif sample_num >=16 and sample_num <=19:
        df_one[key] = 4
    elif sample_num >=20 and sample_num <=23:
        df_one[key] = 5
    
    result1 = pd.concat([result1,df_one])

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 18.33it/s]


In [10]:
# 데이터 카피
train_fin = result.copy()
test_fin = result1.copy()

In [11]:
# 라벨인코딩아닌 더미화 진행
train_fin = pd.get_dummies(result)
test_fin = pd.get_dummies(result1)

In [12]:
x_train = train_fin.drop(['target'], axis=1)

y_train = train_fin['target'] 

print(x_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 24)
(4701217,)
(291241, 16)


In [13]:
## 모델링
# pycaret 을 통한 성능 좋은 모델로 스코어 확인하기

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [15]:
# 모델 불러오기
rf = RandomForestRegressor(random_state=42)

# 모델 학습하기
rf.fit(X_train,Y_train)

# 결과 예측하기
predict = rf.predict(X_test)

# 스코어 확인하기

print(mean_squared_error(predict,Y_test))
print(mean_squared_error(predict,Y_test)**0.5)

48.0304161819444
6.930397981497484


In [16]:
# 모델 불러오기
lgb = LGBMRegressor(random_state=42)

# 모델 학습하기
lgb.fit(X_train,Y_train)

# 결과 예측하기
predict1 = lgb.predict(X_test)

# 스코어 확인하기

print(mean_squared_error(predict1,Y_test))
print(mean_squared_error(predict1,Y_test)**0.5)

54.36688945634755
7.3733906350028375


In [17]:
# submission['target'] = pred
# submission.to_csv("./submit_lgbm.csv", index = False)